# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [4]:
type_dropdown = Select(driver.find_element_by_name('pht_status'))
type_dropdown.select_by_visible_text('Cosmetologists')

In [5]:
text_input = driver.find_element_by_name('pht_lnm')
text_input.send_keys('Nguyen')

In [6]:
button = driver.find_element_by_name('B1')
button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [11]:
# sliced [1:11] to get 10 results and skip column headings
for result in driver.find_elements_by_tag_name('tr')[1:11]:
    print(result.text)

NGUYEN, TOAN HUU
City: SAN ANTONIO
County: BEXAR
Zip Code: 78217


License #(s): 780948, 1706491, 1699123

Complaint # COS20180004289 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $500. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
NGUYEN, HANH CONG
City: EL PASO
County: EL PASO
Zip Code: 79934


License #: 737708

Complaint # COS20180006594 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $1,000. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
NGUYEN, KHIEM VAN
City: LONGVIEW
County: GREGG
Zip Code: 75604


License #: 731665

Complaint # COS20180000257 Date: 5/17/2018

Respondent is assessed an administrative penalty in the amount of $1,250. Respondent failed to follow whirlpool foot spas 

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [21]:
### THE FIRST ONE IS THE COLUMN HEADINGS
results = driver.find_elements_by_tag_name('tr')[:10]
for result in results:
    try:
        mini_results = result.find_elements_by_class_name('results_text')
        print(mini_results[0].text)
    except:
        pass
        #print("It didn't work")

NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, TRUNG N


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [30]:
results = driver.find_elements_by_tag_name('tr')
for result in results[1:11]:
    try:
        cells = result.find_elements_by_tag_name('td')
        print(cells[2].text)
    except:
        # pass
        print("IT DIDN'T WORK!!!!!")

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implements, an

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [37]:
for result in results[1:11]:
    first_cell = result.find_elements_by_tag_name('td')[0]
    mini_results = first_cell.find_elements_by_class_name('results_text')
    print(mini_results[-1].text)

COS20180004289
COS20180006594
COS20180000257
COS20180004915
COS20180009255
COS20140018343
COS20180008846
COS20180000897
COS20170023893
COS20180004076


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [59]:
results = driver.find_elements_by_tag_name('tr')
list_of_dict = []

for result in results[1:]:
    try:
        row = {}
        
        cells = result.find_elements_by_tag_name('td')
        mini_results = result.find_elements_by_class_name('results_text')

        row['Name'] = mini_results[0].text
        
        row['City'] = mini_results[2].text
    
        row['Violation_No'] = mini_results[-2].text
        
        row['Violation_Desc'] = cells[2].text
        
        row['License_No'] = mini_results[-3].text
        
        row['Zip'] = mini_results[3].text
        
        row['County'] = mini_results[2].text
        
        row['City'] = mini_results[1].text
        
        list_of_dict.append(row)
    
    except:
        pass
list_of_dict    

[{'Name': 'NGUYEN, TOAN HUU',
  'City': 'SAN ANTONIO',
  'Violation_No': 'COS20180004289',
  'Violation_Desc': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.',
  'License_No': '780948, 1706491, 1699123',
  'Zip': '78217',
  'County': 'BEXAR'},
 {'Name': 'NGUYEN, HANH CONG',
  'City': 'EL PASO',
  'Violation_No': 'COS20180006594',
  'Violation_Desc': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.',
  'License_No': '737708',
  'Zip': '79934',
  'County': 'EL PASO'},
 {'Name': 'NGUYEN, KHIEM VAN',
  'City': 'LONGVIEW',
  'Violation_No': 'COS20180000257',
  'Violation_Desc': 'Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after

### Save that to a CSV

- Tip: You'll want to use pandas here

In [61]:
import pandas as pd

In [62]:
df = pd.DataFrame(list_of_dict)

In [63]:
df.to_csv("TX_cosmetology_violation.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [64]:
df = pd.read_csv("TX_cosmetology_violation.csv")
df.head()

,City,County,License_No,Name,Violation_Desc,Violation_No,Zip
0,SAN ANTONIO,BEXAR,"780948, 1706491, 1699123","NGUYEN, TOAN HUU",Respondent failed to clean and sanitize whirlp...,COS20180004289,78217
1,EL PASO,EL PASO,737708,"NGUYEN, HANH CONG",Respondent failed to clean and sanitize whirlp...,COS20180006594,79934
2,LONGVIEW,GREGG,731665,"NGUYEN, KHIEM VAN",Respondent failed to follow whirlpool foot spa...,COS20180000257,75604
3,HOUSTON,HARRIS,"1347649, 760528","NGUYEN, DIEP THI NGOC","Respondent failed to disinfect tools, implemen...",COS20180004915,77014
4,SAN ANTONIO,BEXAR,767339,"NGUYEN, LAN T-THUY","Respondent failed to clean, disinfect, and ste...",COS20180009255,78255
